# DATAQUEST 

In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

## Importing libraries 

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
import re
import seaborn as sns

from sklearn import metrics

## Setting some lists to measure progress 

In [3]:
train_map = []
test_map = []

## Getting data using pandas 

In [4]:
PATH = './data/'

In [5]:
!ls {PATH}

sample_submission.csv  test.csv  train.csv


In [6]:
train_raw = pd.read_csv(f'{PATH}train.csv',low_memory=False,parse_dates=['opened_at'])

In [7]:
test_raw = pd.read_csv(f'{PATH}test.csv',low_memory=False,parse_dates=['opened_at'])

##  Fuctions

In [8]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [9]:
def print_score(m):
    print(metrics.mean_absolute_error(m.predict(X_train),y_train),metrics.mean_absolute_error(m.predict(X_valid),y_valid))

In [10]:
def put_in_metrics(n):
    train_map.append(metrics.mean_absolute_error(n.predict(X_train),y_train))
    test_map.append(metrics.mean_absolute_error(n.predict(X_valid),y_valid))

In [11]:
def split_vals(a,n): return a[:n].copy() , a[n:].copy()

n_valid = 3000 
n_trn = len(train) - n_valid
X_train , X_valid = split_vals(train,n_trn)
y_train , y_valid = split_vals(y,n_trn)

X_train.shape , X_valid.shape , y_train.shape , y_valid.shape

NameError: name 'train' is not defined

## EDA (DATA PREPROCESSING) 

In [ ]:
train_raw['target_days'].describe()

In [ ]:
sns.distplot(train_raw['target_days'])

In [ ]:
print('Skewness %f' % train_raw['target_days'].skew())
print('Kurtosis %f' % train_raw['target_days'].kurt())

In [ ]:
train_raw.shape , test_raw.shape

In [ ]:
train_raw.dtypes

In [ ]:
train_raw.head().T

##  Dropping columns

In [ ]:
train_raw.drop(columns=['reassignment_count','reopen_count','Id','vendor'],inplace=True)

## Removing outliers

#### Outliers in update_count, opened_by

In [ ]:
train_raw.drop(train_raw[train_raw['update_count'] !=0].index,inplace=True)

In [ ]:
train_raw.update_count.value_counts()

####  The columns opened_by,location,category,subcategory can be made integer directly by stripping away the alphacharacters .  The columns impact,urgency,priority could be splitted.

In [ ]:
add_datepart(train_raw,'opened_at')

## Splitting column data 

In [ ]:
train_raw['opened_by'] = train_raw['opened_by'].str.split(expand=True)[2]
# train_raw.opened_by.astype('int')

In [ ]:
train_raw['opened_by'].isnull().sum()

In [ ]:
train_raw.dtypes

In [ ]:
# train_raw.opened_by.astype('int',errors='ignore')

In [ ]:
# train_raw['opened_by_na'] = train_raw['opened_by'].copy().isna()

In [ ]:
# train_raw[train_raw['opened_by'].isna()] = 0 

In [ ]:
# train_raw.opened_by_na.value_counts()

#### Now since we have many null values and we got to deal with that. We could do one hot encoding . Make a column of opened_by_na and then give 1 to value having NaN values and 0 to rest . And then in opened_by column we could make all of them as 0. Maybe this is all is pre implemented in a function in fastai . Got to check that. 

In [ ]:
train_cats(train_raw)

In [ ]:
train_raw.impact.cat.set_categories(['3 - Low','2 - Medium','1 - High'],ordered=True,inplace=True)
train_raw.urgency.cat.set_categories(['3 - Low','2 - Medium','1 - High'],ordered=True,inplace=True)
train_raw.priority.cat.set_categories(train_raw.priority.cat.categories[::-1],ordered=True,inplace=True)

In [ ]:
train_raw.isnull().sum().sort_index()/len(train_raw)

# Preprocessing

In [ ]:
def preprocessing(df_raw):
    df = df_raw.copy()
    df.drop(columns=['reassignment_count','reopen_count','Id','vendor'],inplace=True)
    add_datepart(df,'opened_at')
    df['opened_by'] = df['opened_by'].str.split(expand=True)[2]
    return df

## Data after preprocessing 

In [ ]:
display_all(train_raw.head().T)

In [ ]:
train , y, nas = proc_df(train_raw,'target_days')

In [ ]:
# m = RandomForestRegressor(n_jobs=-1,random_state=3)
# m.fit(X_train,y_train)

In [ ]:
# print_score(m)

In [ ]:
# n = RandomForestClassifier(n_estimators=200,n_jobs=-1,random_state=5)
# n.fit(X_train,y_train)

In [ ]:
# print_score(n)

In [ ]:
#  put_in_metrics(n)

In [ ]:
# train_map , test_map

# TEST 

In [ ]:
test = preprocessing(test_raw)

In [ ]:
test.head().T

In [ ]:
apply_cats(test,train)

In [ ]:
test_final,_,_=proc_df(test)

In [ ]:
test_final.shape , train.shape

In [ ]:
# test_final.to_csv('test_final.csv',header=True,index=False)

In [ ]:
# Id = test_raw['Id'].copy()

In [ ]:
# test_raw.drop(columns=['Id','vendor'],inplace=True)

In [ ]:
# add_datepart(test_raw,'opened_at')

In [ ]:
# apply_cats(test_raw,train_raw)

In [ ]:
# test , _, _ = proc_df(test_raw)

In [ ]:
# y_test = n.predict(test).astype('int')

In [ ]:
# submit.to_csv('submission/submit2.csv',header=True,index=False)